In [15]:
import torch
import numpy as np

from itertools import product
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [2]:
# Define the possible actions for each person
actions = ["abs", "wpc", "wopc", "l", "m", "gw", "gow"]

# Generate all possible states for 3 users
states = list(product(actions, repeat=3))

# Display the generated states
print(f"Total states: {len(states)}")
for index, state in enumerate(states):
    print(f"S{index}: {state}")

Total states: 343
S0: ('abs', 'abs', 'abs')
S1: ('abs', 'abs', 'wpc')
S2: ('abs', 'abs', 'wopc')
S3: ('abs', 'abs', 'l')
S4: ('abs', 'abs', 'm')
S5: ('abs', 'abs', 'gw')
S6: ('abs', 'abs', 'gow')
S7: ('abs', 'wpc', 'abs')
S8: ('abs', 'wpc', 'wpc')
S9: ('abs', 'wpc', 'wopc')
S10: ('abs', 'wpc', 'l')
S11: ('abs', 'wpc', 'm')
S12: ('abs', 'wpc', 'gw')
S13: ('abs', 'wpc', 'gow')
S14: ('abs', 'wopc', 'abs')
S15: ('abs', 'wopc', 'wpc')
S16: ('abs', 'wopc', 'wopc')
S17: ('abs', 'wopc', 'l')
S18: ('abs', 'wopc', 'm')
S19: ('abs', 'wopc', 'gw')
S20: ('abs', 'wopc', 'gow')
S21: ('abs', 'l', 'abs')
S22: ('abs', 'l', 'wpc')
S23: ('abs', 'l', 'wopc')
S24: ('abs', 'l', 'l')
S25: ('abs', 'l', 'm')
S26: ('abs', 'l', 'gw')
S27: ('abs', 'l', 'gow')
S28: ('abs', 'm', 'abs')
S29: ('abs', 'm', 'wpc')
S30: ('abs', 'm', 'wopc')
S31: ('abs', 'm', 'l')
S32: ('abs', 'm', 'm')
S33: ('abs', 'm', 'gw')
S34: ('abs', 'm', 'gow')
S35: ('abs', 'gw', 'abs')
S36: ('abs', 'gw', 'wpc')
S37: ('abs', 'gw', 'wopc')
S38: ('ab

In [6]:
state_indices = {state: f"S{index}" for index, state in enumerate(states)}
state_indices

{('abs', 'abs', 'abs'): 'S0',
 ('abs', 'abs', 'wpc'): 'S1',
 ('abs', 'abs', 'wopc'): 'S2',
 ('abs', 'abs', 'l'): 'S3',
 ('abs', 'abs', 'm'): 'S4',
 ('abs', 'abs', 'gw'): 'S5',
 ('abs', 'abs', 'gow'): 'S6',
 ('abs', 'wpc', 'abs'): 'S7',
 ('abs', 'wpc', 'wpc'): 'S8',
 ('abs', 'wpc', 'wopc'): 'S9',
 ('abs', 'wpc', 'l'): 'S10',
 ('abs', 'wpc', 'm'): 'S11',
 ('abs', 'wpc', 'gw'): 'S12',
 ('abs', 'wpc', 'gow'): 'S13',
 ('abs', 'wopc', 'abs'): 'S14',
 ('abs', 'wopc', 'wpc'): 'S15',
 ('abs', 'wopc', 'wopc'): 'S16',
 ('abs', 'wopc', 'l'): 'S17',
 ('abs', 'wopc', 'm'): 'S18',
 ('abs', 'wopc', 'gw'): 'S19',
 ('abs', 'wopc', 'gow'): 'S20',
 ('abs', 'l', 'abs'): 'S21',
 ('abs', 'l', 'wpc'): 'S22',
 ('abs', 'l', 'wopc'): 'S23',
 ('abs', 'l', 'l'): 'S24',
 ('abs', 'l', 'm'): 'S25',
 ('abs', 'l', 'gw'): 'S26',
 ('abs', 'l', 'gow'): 'S27',
 ('abs', 'm', 'abs'): 'S28',
 ('abs', 'm', 'wpc'): 'S29',
 ('abs', 'm', 'wopc'): 'S30',
 ('abs', 'm', 'l'): 'S31',
 ('abs', 'm', 'm'): 'S32',
 ('abs', 'm', 'gw'): 'S

In [ ]:
class FiniteActionStateMachine:
    def __init__(self, states, initial_state):
        self.states = states
        self.current_state = initial_state
        self.transitions = {}

    def add_transition(self, from_state, to_state, condition):
        """Adds a transition between states based on a condition."""
        if from_state not in self.transitions:
            self.transitions[from_state] = []
        self.transitions[from_state].append((condition, to_state))

    def get_next_state(self, input_data):
        """Finds and executes the next state based on input data."""
        if self.current_state in self.transitions:
            for condition, next_state in self.transitions[self.current_state]:
                if condition(input_data):
                    return next_state
        return self.current_state

    def transition(self, input_data):
        """Performs a transition to the next state."""
        next_state = self.get_next_state(input_data)
        print(f"Transitioning from {self.current_state} to {next_state}")
        self.current_state = next_state

In [8]:
# Define mapping of states to descriptive words
state_descriptions = {
    "abs": "absence",
    "wpc": "working with PC",
    "wopc": "working without PC",
    "l": "lunch",
    "m": "meeting",
    "gw": "go to work",
    "gow": "get off work"
}

In [16]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

NameError: name 'SentenceTransformer' is not defined